### Restサーバによるスコアリングの実施
クライアントからのWeb API（Driverless AIサーバ上にREST APIを構築）への接続例

---

Python "Requests"での実行  
[Requests](https://requests-docs-ja.readthedocs.io/en/latest/)

In [39]:
import requests
import json
import pandas as pd

In [10]:
# APIポイント
api_point = "http://54.82.232.159:9090/model/score"
# header
headers = {'Content-Type': 'application/json'}

In [11]:
# カラム定義
columns = ["x1","x2","x3","x4"]

In [47]:
## 1行スコアリング

data1 = [[0.1, 2.3, 7.9, -3.6]]    # Single row

data = {"fields":columns, "rows":data1}

response = requests.post(url=api_point, headers=headers, data=json.dumps(data))
response.json()

{'id': '62c31e2a-635f-11eb-b18a-0242ac110002',
 'fields': ['y'],
 'score': [['2.585485339164734']]}

In [48]:
## 複数行スコアリング

data2 = [[0.1, 2.3, 7.9, -3.6], [0, 0, 0, 0], [2, 5, -8.3, 0.2]]    # multiple rows

data = {"fields":columns, "rows":data2}

response = requests.post(url=api_point, headers=headers, data=json.dumps(data))
response.json()

{'id': '62c31e2a-635f-11eb-b18a-0242ac110002',
 'fields': ['y'],
 'score': [['2.585485339164734'],
  ['0.9894463221232096'],
  ['5.055633544921875']]}

In [49]:
# json形式のインプットデータ
json.dumps(data)

'{"fields": ["x1", "x2", "x3", "x4"], "rows": [[0.1, 2.3, 7.9, -3.6], [0, 0, 0, 0], [2, 5, -8.3, 0.2]]}'

#### pandas DataFrameからの実施

In [41]:
df = pd.read_csv('sample_scoring_data.csv')
df

,x1,x2,x3,x4
0,2.60,-3.21,0.62,1.36
1,0.00,10.00,0.30,1.00
2,-1.11,-1.11,2.22,2.22
3,0.00,0.00,0.00,0.00
4,-9.70,-0.40,6.98,2.10


In [50]:
data3 = [[row[0], row[1], row[2], row[3]] for index, row in df.iterrows()]
data3

[[2.6, -3.21, 0.62, 1.36],
 [0.0, 10.0, 0.3, 1.0],
 [-1.11, -1.11, 2.22, 2.22],
 [0.0, 0.0, 0.0, 0.0],
 [-9.7, -0.4, 6.98, 2.1]]

In [53]:
data = {"fields":columns, "rows":data3}

response = requests.post(url=api_point, headers=headers, data=json.dumps(data))
response.json()

{'id': '62c31e2a-635f-11eb-b18a-0242ac110002',
 'fields': ['y'],
 'score': [['2.006951332092285'],
  ['3.232259194056193'],
  ['-0.8104969635605812'],
  ['0.9894463221232096'],
  ['0.6018870572249094']]}

In [66]:
# 元データとスコアリング結果を結合
predict = [float(y[0]) for y in response.json()['score']]
df["predict"] = predict
df

,x1,x2,x3,x4,predict
0,2.60,-3.21,0.62,1.36,2.006951
1,0.00,10.00,0.30,1.00,3.232259
2,-1.11,-1.11,2.22,2.22,-0.810497
3,0.00,0.00,0.00,0.00,0.989446
4,-9.70,-0.40,6.98,2.10,0.601887
